In [ ]:
import os
import random
import time
import glob
import json
import dit
import itertools
import time
import numpy as np
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns;sns.set()
import pandas as pd
from scipy.optimize import minimize
os.chdir("../")
from helpers.group_helpers import loadsyms, classifylowerorders, classifyoversized
from helpers.group_helpers import classify_syms, srv_to_mat, getcond, oversizedsyms
from helpers.compare_helpers import appendtoPXS,load_frame,addbestsym,normcondentropy
from jointpdfpython3.measures import synergistic_entropy_upper_bound, append_random_srv
from jointpdfpython3.params_matrix import params2matrix_incremental,matrix2params_incremental
from jointpdfpython3.JointProbabilityMatrix import JointProbabilityMatrix
from syndisc.syndisc import self_disclosure_channel
os.chdir("./plot_notebooks")

## Load syms and cluster by SRV type

In [ ]:
states = 2
lenX = 2
subjects=np.arange(lenX)
symss = []
data = {'totmi':[],'indivmi':[],'symsort':[],'states':[]}

concsyms, syms = loadsyms(states)
syms = classifyoversized(syms,states)
if 'lower order' in syms.keys():
    syms = classifylowerorders(states,syms)

listsyms = []
symids = {}
previd = 0
for k in syms.keys():
    for s in syms[k]:
        listsyms.append(s)
        symids[k] = np.arange(previd,previd+len(syms[k]))
    previd = previd+len(syms[k])

## Compute cost of each constructed SRV given X

In [ ]:
def getcosts(row,syms):
    costs = []
    lenjXS = int(row['lenX'])
    upper = row['syn_upper']
    print(row['systemID'])
    for i,s in enumerate(syms):
        pX = row['pX'].copy()
        pXSSym = appendtoPXS(lenX,pX,pX,s)
        d = dit.Distribution.from_ndarray(pXSSym)
        totmi = dit.shannon.mutual_information(d,subjects,[lenjXS])
        indivmi = sum([dit.shannon.mutual_information(d,[j],[lenjXS]) for j in subjects])
        wms = totmi-indivmi
        cost = (upper - wms)/upper
        if totmi!=0:
            cost+=(indivmi/totmi)
        else:
            cost+=indivmi
        costs.append(cost)
    row['costs'] = costs
    return row

d = pd.read_pickle("../../results/nprandomdirichlet2345.pkl")
d = d[d['states']==states].head(100)
d = load_frame(d=d)
plot = d.apply(lambda row:getcosts(row,listsyms),axis=1)
plot.to_pickle("../../results/test/histbestsymXstates"+str(states)+".pkl") 

## LOAD GENERATED HISTDATA

In [ ]:
# hist of top5 classes 
unique5=False
costs = plot['costs'].values
top = 5
histdata = {n:[] for n in np.arange(top)+1}
for cost in costs:
    arrconds = []
    for i,c in enumerate(cost):
        kx = [k for k,v in symids.items() if i in v][0]
        arrconds.append([kx,c])
    arr = np.array(arrconds)
    arr = arr[arr[:, 1].argsort()]
    
    if unique5:
        # only keep top 5 unique values
        top5 = arr[np.unique(arr[:,1], return_index=True)[1]][:top]
    else:
        top5=arr[:top]
    for i,c in enumerate(top5[:,0]):
        histdata[i+1].append(c)
        
histd = pd.DataFrame(data=histdata)
histd = histd.melt(value_vars=list(histdata.keys()),
              var_name='Place', value_name='Class')

sns.set_context("paper", font_scale = 1.8, rc={"legend.fontsize": 13})
g=sns.histplot(data=histd,x='Class',hue='Place',multiple='dodge',palette='tab10')
plt.xticks(rotation=25)
plt.title('Top 5 lowest cost '+"States = "+str(states))
fig = g.get_figure()
plt.show()

## Compute for each constructed SRVs S the conditional entropy H(Sfound|S) and best of 5

In [ ]:
def bestoffive(row,syms,top=5,lenX=2):
    print(row['systemID'],datetime.fromtimestamp(time.time()))
    conds = row['H(Sfound|Sym)']
    cond_top5 = sorted(zip(np.arange(len(conds)),conds), key=lambda t: t[1])[:top]
    bestids = np.array(cond_top5,dtype=int)[:,0]
    sym_top5 = list(map(syms.__getitem__,bestids))
    jointp = row['pXS'].copy()
    for s in sym_top5:
        jointp = appendtoPXS(lenX,row['pX'],jointp,s)
    ditd = dit.Distribution.from_ndarray(jointp)
    synvars = list(range(lenX+1,len(ditd.rvs)))
    row['H(Sfound|bestof)'] = dit.shannon.conditional_entropy(ditd,[lenX],synvars)/row['H(S)']
    row['H(Sfound|Smin)'] = cond_top5[0][1]
    return row
    
df = pd.read_pickle("../../results/test/histbestsymXstates"+str(states)+".pkl")
df = d.apply(lambda row:normcondentropy(row,listsyms),axis=1)
df = df.apply(lambda row:bestoffive(row,listsyms),axis=1)
df.to_pickle("../../results/test/histbestsymSfoundstates"+str(states)+".pkl") 
plot=pd.read_pickle("../../results/test/histbestsymSfoundstates"+str(states)+".pkl")
plot = plot.head(100)

In [ ]:
df = plot
allconds = df['H(Sfound|Sym)'].values
histdata = {n:[] for n in np.arange(5)+1}
unique5=False
for conds in allconds:
    arrconds = []
    for i,c in enumerate(conds):
        kx = [k for k,v in symids.items() if i in v][0]
        arrconds.append([kx,c])
    arr = np.array(arrconds)
    arr = arr[arr[:, 1].argsort()]
    
    if unique5:
        # only keep top 5 unique values
        top5 = arr[np.unique(arr[:,1], return_index=True)[1]][:5]
    else:
        top5=arr[:5]
    for i,c in enumerate(top5[:,0]):
        histdata[i+1].append(c)
        
histd = pd.DataFrame(data=histdata)
histd = histd.melt(value_vars=list(histdata.keys()),
              var_name='Place', value_name='Class')

sns.set_context("paper", font_scale = 1.8, rc={"legend.fontsize": 13})
g=sns.histplot(data=histd,x='Class',hue='Place',multiple='dodge',palette='tab10')
plt.xticks(rotation=15)
plt.title('Top 5 lowest H(Sfound|Sym)'+" States = "+str(states))
fig = g.get_figure()
plt.show()
